# Introduction

Ici je vais écrire la version finale de la méthode supervisée pour le *cas 1 : Appariement connu*


1. [Loading of datasets](#Load-datasets)
2. [Transformation of datasets](#Transform-datasets)
3. [Helper functions](#Helper-functions)
4. [Linear](#Linear)
    1. [Generate-data](#Generate-data)
    2. [Prepare datasets](#Prepare-datasets)
    5. [Neural Network Architecture](#Neural-Network-Architecture)
    6. [Compile the NN](#Compile-the-NN)
    7. [Train the NN](#Train-the-NN)
5. [NN reverse](#NN-reverse)
    1. [Source data](#Source-data)
    2. [Target data](#Target-data)
    5. [Aligmnent](#Aligmnent)
    6. [NN archi](#NN-archi)
    7. [Training](#Training)
    8. [Visualization](#Visualization)
6. [Remaining work](#Remaining-work)

[**[Back to top]**](#Introduction)

In [ ]:
from __future__ import division, print_function
import sys
if '..' not in sys.path:
    sys.path.append('..')

import theano
import theano.tensor as T
import lasagne

import time
import visual

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from sklearn.metrics import confusion_matrix, pairwise_distances

from nn.helper import CNN, NN
from nn import block as nnb
from nn import compilers as nnc
from nn import rgl

In [ ]:
%matplotlib inline

# Load datasets

- the datasets are loaded/built.
- The batchsize is defined
- half of the data name (the source part) is defined

[**[Back to top]**](#Introduction)

## Datasets Imports 

In [ ]:
from datasets.toys import make_clouds, make_circles, make_X, make_moons
from datasets.utils import make_dataset, make_domain_dataset
from datasets.utils import shuffle_array


# Transform datasets

- the transformed datasets are built.
- last part of the data name (the target part) is defined

[**[Back to top]**](#Introduction)

## Transformation Imports

In [ ]:
from datasets.utils import make_domain_dataset, make_corrector_dataset
import datasets.transform as transform

# Helper functions

[**[Back to top]**](#Introduction)

In [ ]:
# Import loggers
from logs import new_logger, empty_logger
logger = new_logger()

In [ ]:
from align_learn.probability import mass, align_tgt, align_src
from align_learn.preprocess import _classwise_shuffle

def classwise_shuffle(data):
    data['X_train'] = _classwise_shuffle(data['X_train'], data['labels'])


In [ ]:
BATCHSIZE = 50

# Linear

[**[Back to top]**](#Introduction)

## Generate data

Première étape : générer les données

[**[Back to top]**](#Introduction)

In [ ]:
# n_classes_1 = 4
# n_classes_2 = 4
n_samples = 1000
X_src, y_src = make_moons(n_samples=n_samples)

X_tgt, y_tgt = transform.rotate(X_src, y_src, angle=15.)
# X_tgt, y_tgt = make_circles(n_samples=n_samples,  n_classes=n_classes_2)

data_name='Clouds -> rotated'

In [ ]:
# labels
# l_src, l_tgt = k_means_src.labels_, k_means_tgt.labels_
l_src, l_tgt = np.asarray(y_src, dtype=int), np.asarray(y_tgt, dtype=int),

# Mass
w_src = mass(l_src)
w_tgt = mass(l_tgt)

# Params
n_class_tgt = len(np.unique(l_tgt))
n_class_src = len(np.unique(l_src))

## Prepare datasets

Build the training datasets.

The data from the source and the target distribution ordered so $x_s$ should correspond to $x_t$.

The target is the probability that $x$ belong to the label $y$ in the source space

[**[Back to top]**](#Introduction)

In [ ]:
# Align the data : No alignment requiered
X_S, y_S = X_src, l_src
X_T, y_T = X_tgt, l_tgt
# Get the probability to be predicted for each couple of data point.

# Shuffle it all to prevent the index to be correlated to the labels
X_S, y_S, X_T, y_T = shuffle_array(X_S, y_S, X_T, y_T)
# Build split dataset (train, valid, test)
src_data = make_dataset(X_S, y_S, batchsize=100)
tgt_data = make_dataset(X_T, y_T, batchsize=100)
corr_data = make_corrector_dataset(src_data, tgt_data)
adversarial_data = make_domain_dataset([src_data, tgt_data])

## Neural Network Architecture

the neural network works as a supervised multiple regression model.

[**[Back to top]**](#Introduction)

In [ ]:
# Get general information :
# =========================
batchsize = src_data.batchsize
_shape = np.shape(src_data.X_train)
n_dim = len(_shape)
n_features = np.prod(_shape[1:])

shape = (batchsize,) + _shape[1:]

# Logs
logger.info('Building the input and output variables for : {}'.format(data_name))
logger.info('Input data expected shape : {}'.format(shape))

# Build the layers :
# ==================
# Inputs layers
# -------------
input_layer_src = lasagne.layers.InputLayer(shape=shape)
input_layer_tgt = lasagne.layers.InputLayer(shape=shape)

# Representaion layers for the source data
# ----------------------------------------
dense_1 = lasagne.layers.DenseLayer(input_layer_src, shape[1], nonlinearity=lasagne.nonlinearities.rectify)
dense_2 = lasagne.layers.DenseLayer(dense_1, shape[1], nonlinearity=lasagne.nonlinearities.rectify)
dense_3 = lasagne.layers.DenseLayer(dense_2, shape[1], nonlinearity=lasagne.nonlinearities.rectify)
dense_4 = lasagne.layers.DenseLayer(dense_3, shape[1], nonlinearity=None)

dense_0 = lasagne.layers.DenseLayer(input_layer_src, shape[1], nonlinearity=None)

end_layer = dense_0


# Adversarial structure
# ---------------------
adv_inputs = [end_layer, input_layer_tgt]
concat = lasagne.layers.ConcatLayer(adv_inputs, axis=0)
rg_layer = rgl.ReverseGradientLayer(concat, hp_lambda=1)
adv_dense_1 = lasagne.layers.DenseLayer(rg_layer, 3, 
                                       nonlinearity=lasagne.nonlinearities.tanh)
adv_dense_2 = lasagne.layers.DenseLayer(adv_dense_1, 3,
                                       nonlinearity=lasagne.nonlinearities.tanh)
adv_output = lasagne.layers.DenseLayer(adv_dense_2, len(adv_inputs), 
                                       nonlinearity=lasagne.nonlinearities.softmax,)

# Adversarial bis for learning only
adv_inputs2 = [end_layer, input_layer_tgt]
concat2 = lasagne.layers.ConcatLayer(adv_inputs2, axis=0)
rg_layer2 = rgl.ReverseGradientLayer(concat2, hp_lambda=0)
adv_dense_12 = lasagne.layers.DenseLayer(rg_layer2, 3, W=adv_dense_1.W, b=adv_dense_1.b,
                                        nonlinearity=lasagne.nonlinearities.tanh)
adv_dense_22 = lasagne.layers.DenseLayer(adv_dense_12, 3, W=adv_dense_2.W, b=adv_dense_2.b,
                                        nonlinearity=lasagne.nonlinearities.tanh)
adv_output2 = lasagne.layers.DenseLayer(adv_dense_22, len(adv_inputs2), W=adv_output.W, b=adv_output.b,
                                       nonlinearity=lasagne.nonlinearities.softmax,)


## Compile the NN

Compile the functions:
- training, validation, proba output for the source path
- training, validation, proba output for the target path
- raw output for the representation
- training, validation, proba output for the adverssarial path


[**[Back to top]**](#Introduction)

In [ ]:
# Instanciate the NN :
# ====================
nn = CNN(name='Linear NN')
nn.add_output('correction', end_layer)
# Ok for the adversarial the code is not intuitive. [Further work]
nn.add_output('adversarial', adv_inputs)
nn.add_output('adversarial2', adv_inputs2)

# Compile :
# =========
nn.compile('correction', nnc.squared_error_sgd_mom, lr=0.01, mom=0.)
nn.compile('correction', nnc.squared_error_validation)
nn.compile('correction', nnc.output)
nn.compile('adversarial', nnc.adversarial, output_layer=adv_output, lr=0.01, mom=0.)
nn.compile('adversarial2', nnc.adversarial, output_layer=adv_output2, lr=0.1, mom=0.8)

logger.info("Compilation Done")

## Train the NN

Now is the training session.

It altarnatively (mini-batch after mini-batch) train (forwward-backward propagation) each part of the neural network.

[**[Back to top]**](#Introduction)

In [ ]:
# Train the nn :
# ==============
# nn.train(data, num_epochs=100);

for _ in range(15):
    classwise_shuffle(corr_data)
#     nn.train([corr_data], ['correction'], num_epochs=1);
    nn.train([corr_data, adversarial_data], ['correction', 'adversarial'], num_epochs=2);
    nn.train([adversarial_data], ['adversarial2'], num_epochs=5);

In [ ]:
# ================
# Learning curve
# ================
# Usefull regex : 'proba.* loss', 'loss', 'acc'
fig, ax = visual.learning_curve(nn.global_stats, regex='corr.* loss')
#     SAVE
# fig.tight_layout()
# fig.savefig(fig_title+'-Learning_curve.png',bbox_inches='tight')
fig.show()

In [ ]:
# ================
# Learning curve
# ================
# Usefull regex : 'proba.* loss', 'loss', 'acc'
fig, ax = visual.learning_curve(nn.global_stats, regex='adv.* loss')
#     SAVE
# fig.tight_layout()
# fig.savefig(fig_title+'-Learning_curve.png',bbox_inches='tight')
fig.show()

In [ ]:
# ================
# Learning curve
# ================
# Usefull regex : 'proba.* loss', 'loss', 'acc'
fig, ax = visual.learning_curve(nn.global_stats, regex='training.* acc')
#     SAVE
# fig.tight_layout()
# fig.savefig(fig_title+'-Learning_curve.png',bbox_inches='tight')
fig.show()

## Check some results

Check the output of the NN:

The predicted probability of being in a partition **vs** the true value.

[**[Back to top]**](#Introduction)

In [ ]:
fig, ax = visual.source_2D(src_data.X_test, src_data.y_test)
visual.target_2D(tgt_data.X_test, tgt_data.y_test, ax=ax);
visual.add_legend(ax, title="Source vs target data")
plt.show()

In [ ]:
X_pred = nn.parts['correction'].output(corr_data.X_test)[0]
fig, ax = visual.target_2D(tgt_data.X_test, tgt_data.y_test)
visual.corrected_2D(X_pred, src_data.y_test, ax=ax);
visual.add_legend(ax, title="Corrected vs target Data")
# plt.yscale('log')
plt.show()

In [ ]:
X_s, X_t = adversarial_data.X_test
y = np.hstack([np.zeros(X_s.shape[0]), np.ones(X_t.shape[0])])
X_pred = nn.parts['correction'].output(X_s)[0]
X = np.concatenate([X_pred, X_t])

from matplotlib import cm
CMAP = 'Paired'
cm_heat = plt.cm.RdBu
color = cm.ScalarMappable(cmap=CMAP)
fig, ax = plt.subplots()
x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
xx, yy = np.meshgrid(np.arange(x_min, x_max, .02),
                     np.arange(y_min, y_max, .02))
c = np.c_[xx.ravel(), yy.ravel()]
Z = nn.parts['adversarial'].output(c[0:5], c[5:])
Z = np.array(Z)[0, :, 0]
# Put the result into a color plot
Z = Z.reshape(xx.shape)
ax.contourf(xx, yy, Z, cmap=cm_heat, alpha=.8)

# Plot also the training points
ax.scatter(X[:, 0], X[:, 1], c=color.to_rgba(y));
ax.set_xlim(xx.min(), xx.max());
ax.set_ylim(yy.min(), yy.max());

# X_pred = nn.parts['correction'].output(corr_data.X_test)[0]
# visual.corrected_2D(X_pred, src_data.y_test, ax=ax)

In [ ]:
Z.shape, np.prod(xx.shape)

# NN reverse

[**[Back to top]**](#Introduction)

## Source data

[**[Back to top]**](#Introduction)

## Target data

[**[Back to top]**](#Introduction)

## Alignment

[**[Back to top]**](#Introduction)

## NN archi

[**[Back to top]**](#Introduction)

## NN compile

[**[Back to top]**](#Introduction)

## Training

[**[Back to top]**](#Introduction)

## Visualization

[**[Back to top]**](#Introduction)

# Remaining work

- **Tout refaire**

[**[Back to top]**](#Introduction)